In [ ]:
%matplotlib inline
import os
import matplotlib.pyplot as plt
import numpy as np
from importlib_resources import open_binary, open_text
import PIL.Image

In [ ]:
data_dir = os.path.realpath(os.path.join('..', 'tests', 'data'))
print(data_dir)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from pose3d_utils.skeleton import MPI3D_SKELETON_DESC, CANONICAL_SKELETON_DESC

group_colours = dict(centre='magenta', left='blue', right='red')

def plot_skeleton_on_axes3d(skel, skel_desc, ax: Axes3D, invert=True, alpha=1.0):
    ax.set_xlabel('x')
    ax.set_ylabel('z')
    ax.set_zlabel('y')

    # NOTE: y and z axes are swapped
    xs = skel.narrow(-1, 0, 1).numpy()
    ys = skel.narrow(-1, 2, 1).numpy()
    zs = skel.narrow(-1, 1, 1).numpy()

    # Correct aspect ratio (https://stackoverflow.com/a/21765085)
    max_range = np.array([
        xs.max() - xs.min(), ys.max() - ys.min(), zs.max() - zs.min()
    ]).max() / 2.0
    mid_x = (xs.max() + xs.min()) * 0.5
    mid_y = (ys.max() + ys.min()) * 0.5
    mid_z = (zs.max() + zs.min()) * 0.5
    ax.set_xlim(mid_x - max_range, mid_x + max_range)
    ax.set_ylim(mid_y - max_range, mid_y + max_range)
    ax.set_zlim(mid_z - max_range, mid_z + max_range)
    ax.set_aspect('equal')

    if invert:
        ax.invert_zaxis()

    # Set starting view
    ax.view_init(elev=20, azim=-100)

    for joint_id, joint in enumerate(skel):
        meta = skel_desc.get_joint_metadata(joint_id)
        color = group_colours[meta['group']]
        parent = skel[meta['parent']]
        offset = parent - joint
        ax.quiver(
            [joint[0]], [joint[2]], [joint[1]],
            [offset[0]], [offset[2]], [offset[1]],
            color=color,
            alpha=alpha,
        )

    ax.scatter(xs, ys, zs, color='grey', alpha=alpha)

def visualise_example(camera, image, points, skel_desc=MPI3D_SKELETON_DESC, save=None):
    """Show the image and projected points in a matplotlib figure."""
    points2d = camera.project_cartesian(torch.as_tensor(points))
    fig = plt.figure(figsize=(16, 8))
    ax1 = fig.add_subplot(1, 2, 1)
    plt.imshow(image)
    for joint_id, (x, y) in enumerate(np.array(points2d)):
        meta = skel_desc.get_joint_metadata(joint_id)
        color = group_colours[meta['group']]
        parent = points2d[meta['parent']]
        ax1.plot([x, parent[0]], [y, parent[1]], color=color)
    ax1.scatter(points2d[:, 0], points2d[:, 1], color='grey', s=12)
    ax1.set_xlim([0, image.width - 1])
    ax1.set_ylim([image.height - 1, 0])
    ax2 = fig.add_subplot(1, 2, 2, projection='3d')
    plot_skeleton_on_axes3d(torch.as_tensor(points), skel_desc, ax2)
    if save:
        plt.savefig(save)
    plt.show()

In [ ]:
import json
from pose3d_utils.camera import CameraIntrinsics
import torch

img = PIL.Image.open(os.path.join(data_dir, 'example01_image.jpg'))

with open(os.path.join(data_dir, 'example01_camera.json'), 'r') as f:
    camera_params = json.load(f)

camera = CameraIntrinsics(torch.tensor(camera_params['intrinsic'])[:3])

with open(os.path.join(data_dir, 'example01_univ_annot3.txt'), 'r') as f:
    univ_annot3 = np.loadtxt(f)

visualise_example(camera, img, univ_annot3)

In [ ]:
from pose3d_utils.transformers import TransformerContext
import pose3d_utils.transforms as transforms

points2d = camera.project_cartesian(torch.as_tensor(univ_annot3))
cx, cy = points2d[MPI3D_SKELETON_DESC.root_joint_id].tolist()

transforms = [
    transforms.PanImage(camera.x_0 - cx, camera.y_0 - cy),
    transforms.RotateImage(30),
    transforms.ZoomImage(2.0),
    transforms.HorizontalFlip(do_flip=True, flip_indices=MPI3D_SKELETON_DESC.hflip_indices),
]

for i in range(len(transforms) + 1):
    ctx = TransformerContext(camera, img.width, img.height, msaa=1)
    for transform in transforms[:i]:
        ctx.add(transform)
    tuple1 = (camera, img, torch.as_tensor(univ_annot3))
    tuple2 = ctx.transform(*tuple1)
    if i > 0:
        print(transforms[i - 1])
    # visualise_example(*tuple2, save='Transformations_{:02d}.svg'.format(i))
    visualise_example(*tuple2)